In [2]:
import pandas as pd
import json
from deepdiff import DeepDiff
import itertools

In [3]:
# Definindo o dicionário com os paths dos arquivos JSON
json_categorias = {
    'br': 'data/BR_category_id.json',
    'gb': 'data/GB_category_id.json',
    'ca': 'data/CA_category_id.json',
    'us': 'data/US_category_id.json'
}

# Criação de uma função para carregar e transformar os arquivos JSON
def carregar_transformar_json(path):
    # Carrega o JSON em um dataframe
    df = pd.read_json(path)
    
    # Expande a coluna 'items' em linhas separadas, pois esta está dentro de um dicionário
    items = pd.json_normalize(df['items'])
    
    return items

# Iterando sobre o dicionário para carregar os DataFrames transformados
df = {}
for key, path in json_categorias.items():
    df[key] = carregar_transformar_json(path)

# Criação de uma lista contendo os dataframes
lista_dfs = [df['br'],df['ca'],df['gb'],df['us']]

In [4]:
# Testando o número de linhas esperado se concatenar todos os dataframes
tamanho_esperado = 0
for df in lista_dfs:
    tamanho_esperado+=df.shape[0]

print(f"Todos os arquivos JSON possuem juntos {tamanho_esperado} registros.")

Todos os arquivos JSON possuem juntos 125 registros.


In [5]:
# Como o dataframe resultante da concatenação terá apenas 125 linhas
#  a concatenação será feita para facilitar a análise
df_json = pd.concat(lista_dfs).reset_index(drop=True)

# Análise do tamanho do dataframe resultante e tipagem das colunas
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   kind                125 non-null    object
 1   etag                125 non-null    object
 2   id                  125 non-null    object
 3   snippet.title       125 non-null    object
 4   snippet.assignable  125 non-null    bool  
 5   snippet.channelId   125 non-null    object
dtypes: bool(1), object(5)
memory usage: 5.1+ KB


In [71]:
# Remoção dos termos "snippet." do nome das colunas oriundas deste dicionário
df_json.rename(columns=lambda col: col.replace('snippet.', ''), inplace=True)

# Análise da contagem de registros por coluna
for col in df_json:
    print(df_json[col].value_counts(), '\n')
    print(f'{"-" * 100}', '\n')

kind
youtube#videoCategory    125
Name: count, dtype: int64 

---------------------------------------------------------------------------------------------------- 

etag
IfWa37JGcqZs-jZeAyFGkbeh6bc    4
5XGylIs7zkjHh5940dsT5862m1Y    4
3IaZjRK8JeIkLMfRDX57P4FFb9A    4
2sKmjip5q7FHJgqkHvnpMDgmddE    4
qJ2eDOZWUDj8fbFYAIG_yzPCYRc    4
HwuNLGhBJ4PZCEurr-SoqvHuKx8    4
3F8eda4xnGzGFmlpsFaf5_YpOgU    4
Z3yra5IR8n2bkjvv3ZheAOQ-tUk    4
H6d6HrYDy3EAVAxQR8Z-ShEva1k    4
7fvdoCRRWE89SpB5UaOY8gGdbk0    4
FD7naICnxxl8HOnZZJAzrWplzuU    4
ZFbZGeeLYWolvmCLQM7m8Yio104    4
M6Y5Av0CP45r5GgpzamLfHj41y4    4
harT7v7A6516SPdkqp81Xy5yfBw    4
LNgRKNynxC50MeYV9e6XLPX9JC4    4
totPMF_82XTvVFy0ExFgoKhTklM    4
tMfbFvkfuPP8YnUEqpr0gcR6aWU    4
KEdEtUd4WGk_aACrYI03UhGE33w    4
hHUhloYhyMMqVkQ4LH9sr3aRQoU    4
gYzt8dB8mlod-84ipMtjynYe9xU    4
IbGXblQi8v_nOsXjirhul1NVcH8    4
Qi1csjh-POReitZEbR7CUw7dmao    4
v2n6q4JttoL4uUbazrcxNQRGPZk    4
QMEBz6mxVdklVaq8JwesPEw_4nI    4
D1W6tq5mMMCV0wtNxf9A6g9wWjU    4
I3IL9

 - Fazendo a análise dos registros por coluna a coincidência de todas as colunas possuírem grupos de 4 registros iguais em cada uma chamou a atenção, será feita uma análise comparativa dos dataframes e caso estejam iguais, será feita análise dos arquivos JSON que deram origem aos dataframes.

In [6]:
# Verificando um id aleatoriamente pois dá a impressão que os 4 JSON são iguais
df_json[df_json.id == '1']

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
31,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
62,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
93,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ


In [7]:
# Gera todas as combinações únicas de pares de dataframes para observar se há diferença entre si
for i, j in itertools.combinations(range(len(lista_dfs)), 2):
    df1 = lista_dfs[i]
    df2 = lista_dfs[j]
    if df1.equals(df2):
        print(f"Os DataFrames df{i} e df{j} são iguais.")
    else:
        print(f"Os DataFrames df{i} e df{j} são diferentes.")

Os DataFrames df0 e df1 são iguais.
Os DataFrames df0 e df2 são iguais.
Os DataFrames df0 e df3 são diferentes.
Os DataFrames df1 e df2 são iguais.
Os DataFrames df1 e df3 são diferentes.
Os DataFrames df2 e df3 são diferentes.


In [8]:
# Função para carregar arquivos JSON usando a lib json
def carregar_json(path):
    with open(path, 'r') as file:
        return json.load(file)

# Dicionário com os paths dos arquivos JSON
json_categorias = {
    'us': 'data/US_category_id.json',
    'br': 'data/BR_category_id.json',
    'gb': 'data/GB_category_id.json',
    'ca': 'data/CA_category_id.json'
}

# Carrega conteúdos dos arquivos
json_data = {key: carregar_json(path) for key, path in json_categorias.items()}

# Função para comparar dois JSONs e imprimir diferenças
def comparar_jsons(json1, json2, key1, key2):
    diff = DeepDiff(json1, json2, ignore_order=True)
    if diff:
        print(f"Diferenças entre {key1} e {key2}:")
        print(diff)
    else:
        print(f"Não há diferenças entre {key1} e {key2}.")

# Comparar cada par de arquivos JSON
chaves = list(json_categorias.keys())
for i in range(len(chaves)):
    for j in range(i + 1, len(chaves)):
        key1 = chaves[i]
        key2 = chaves[j]
        comparar_jsons(json_data[key1], json_data[key2], key1, key2)


Diferenças entre us e br:
{'values_changed': {"root['etag']": {'new_value': 'kBCr3I9kLHHU79W4Ip5196LDptI', 'old_value': 'HIrK3n45Uw2IYz9_U2-gK1OsXvo'}}, 'iterable_item_removed': {"root['items'][16]": {'kind': 'youtube#videoCategory', 'etag': 'AioSVwhKNpZ2bhtw9SJmrkfY5-I', 'id': '29', 'snippet': {'title': 'Nonprofits & Activism', 'assignable': True, 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}}}}
Diferenças entre us e gb:
{'values_changed': {"root['etag']": {'new_value': 'kBCr3I9kLHHU79W4Ip5196LDptI', 'old_value': 'HIrK3n45Uw2IYz9_U2-gK1OsXvo'}}, 'iterable_item_removed': {"root['items'][16]": {'kind': 'youtube#videoCategory', 'etag': 'AioSVwhKNpZ2bhtw9SJmrkfY5-I', 'id': '29', 'snippet': {'title': 'Nonprofits & Activism', 'assignable': True, 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}}}}
Diferenças entre us e ca:
{'values_changed': {"root['etag']": {'new_value': 'kBCr3I9kLHHU79W4Ip5196LDptI', 'old_value': 'HIrK3n45Uw2IYz9_U2-gK1OsXvo'}}, 'iterable_item_removed': {"root['items'][16]": {'kind': '

**Os arquivos ``BR_category_id.json`` ``GB_category_id.json`` ``CA_category_id.json`` são exatamente iguais e a única diferença entre eles e o arquivo ``US_category_id.json`` é a chave principal (root) "etag"**

É provável que as análises relacionadas a categorização dos vídeos seja comprometida em virtude da falta de dados relacionados a esta!

In [10]:
# Dropando as linhas duplicadas do dataframe para reduzir 
# a dimensão do mesmo e evitar posteriores problemas de join
df_json.drop_duplicates(inplace=True, ignore_index=True)

In [18]:
# Define o nome da variável que irá armezar o dataframe na chave, 
# e o path do arquivo no valor
dfs = {'br':'data/BR_youtube_trending_data.csv',
       'us':'data/US_youtube_trending_data.csv',
       'ca':'data/CA_youtube_trending_data.csv',
       'gb':'data/GB_youtube_trending_data.csv'}

dfs_dataframes = {}
lista_dfs_csv = []

# Iterando sobre os items da lista para criar os dataframes
for country, path in dfs.items():
    # Lê o arquivo CSV
    df = pd.read_csv(path)
    # Adiciona uma coluna 'country' com o nome do país
    df['country'] = country
    # Transforma a sigla no nome do país
    df['country'] = df['country'].map({'br':'Brasil',
                                       'us':'Estados Unidos',
                                       'ca':'Canada',
                                       'gb':'Reino Unido'})
    # Atribui o dataframe a uma variável global com o nome do país
    dfs_dataframes[country] = df
    lista_dfs_csv.append(dfs_dataframes[country])

In [19]:
tamanho_esperado_csv = 0
memoria_usada_total = 0
for i, file in enumerate(lista_dfs_csv):
    memoria_usada = round(file.memory_usage().sum() / (1024**2), 1)
    print(f"O {i + 1}o arquivo possui shape {file.shape[0]:,} linhas e {file.shape[1]} colunas. -> {memoria_usada} MB")
    tamanho_esperado_csv += file.shape[0]
    memoria_usada_total += memoria_usada

print(f"\nO tamanho total esperado do dataframe é {tamanho_esperado_csv:,} linhas -> {memoria_usada_total:.1f} MB.")

O 1o arquivo possui shape 63,193 linhas e 17 colunas. -> 7.4 MB
O 2o arquivo possui shape 57,191 linhas e 17 colunas. -> 6.7 MB
O 3o arquivo possui shape 57,144 linhas e 17 colunas. -> 6.6 MB
O 4o arquivo possui shape 57,195 linhas e 17 colunas. -> 6.7 MB

O tamanho total esperado do dataframe é 234,723 linhas -> 27.4 MB.


 - Ainda que o dataframe fique grande em termos de linhas, o espaço em memória ocupado será pequeno e a concatenação é válida pois facilitará a análise.

In [21]:
# Concatena os 4 dataframes e reseta o index para facilitar a análise
df_csv = pd.concat(lista_dfs_csv).reset_index(drop=True)

In [23]:
# Verificando o resultado da concatenação
df_csv

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,country
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,22,2020-08-12T00:00:00Z,pietro|guedes|ingrid|ohara|pingrid|vlog|amigos...,263835,85095,487,4500,https://i.ytimg.com/vi/s9FH4rDMvds/default.jpg,False,False,"Salve rapaziada, neste vídeo me declarei pra e...",Brasil
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,10,2020-08-12T00:00:00Z,JYP Entertainment|JYP|ITZY|있지|ITZY Video|ITZY ...,6000070,714310,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,Brasil
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10T14:59:00Z,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,22,2020-08-12T00:00:00Z,OH JULIANA PARÓDIA|MC Niack PARÓDIA|PARÓDIAS|A...,2296748,39761,5484,0,https://i.ytimg.com/vi/3EfkCrXKZNs/default.jpg,True,False,Se inscrevam meus amores! 📬 Quer nos mandar al...,Brasil
3,gBjox7vn3-g,Contos de Runeterra: Targon | A Estrada Tortuosa,2020-08-11T15:00:09Z,UC6Xqz2pm50gDCORYztqhDpg,League of Legends BR,20,2020-08-12T00:00:00Z,Riot|Riot Games|League of Legends|lol|trailer|...,300510,46222,242,2748,https://i.ytimg.com/vi/gBjox7vn3-g/default.jpg,False,False,Você se unirá aos Lunari e aos Solari em Targo...,Brasil
4,npoUGx7UW7o,Entrevista com Thammy Miranda | The Noite (10/...,2020-08-11T20:04:02Z,UCEWOoncsrmirqnFqxer9lmA,The Noite com Danilo Gentili,23,2020-08-12T00:00:00Z,The Noite|The Noite com Danilo Gentili|Danilo ...,327235,22059,3972,2751,https://i.ytimg.com/vi/npoUGx7UW7o/default.jpg,False,False,Danilo Gentili recebe Thammy Miranda. Após pas...,Brasil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234718,fiWxNzG5rOw,I Cheated with ONE WAY GLASS in a Building Com...,2021-05-24T04:48:40Z,UCzMjRlKVO9XIqH_crIFpi6w,Skeppy,20,2021-06-01T00:00:00Z,minecraft|mine craft|minecraft youtuber|pg|fam...,2295910,135826,1605,7583,https://i.ytimg.com/vi/fiWxNzG5rOw/default.jpg,False,False,"Skeppy & BadBoyHalo Troll Antfrost, RedVelvet,...",Reino Unido
234719,z1mcEjHobr8,STEADY! Headie One - Daily Duppy | GRM Daily -...,2021-05-25T18:00:15Z,UCDf1u4zE618cbWE7Qt3eNWQ,Denz&Renz,10,2021-06-01T00:00:00Z,[None],41397,1906,12,146,https://i.ytimg.com/vi/z1mcEjHobr8/default.jpg,False,False,A Steady Duppy.LINK TO ORIGINAL VIDEO: https:/...,Reino Unido
234720,-Ot0vHUyEpw,The Weeknd - The Weeknd – Save Your Tears (Liv...,2021-05-24T04:52:51Z,UCF_fDSgPpBQuh1MsUTgIARQ,TheWeekndVEVO,10,2021-06-01T00:00:00Z,The Weeknd|Weeknd|The Weekend|Weekend|bbmas|bi...,9000398,554138,3922,27866,https://i.ytimg.com/vi/-Ot0vHUyEpw/default.jpg,False,False,“Save Your Tears” (Live at The 2021 Billboard ...,Reino Unido
234721,t-Y-1Ki6LEk,ETERNALS Trailer: Every EASTER EGG + Captain ...,2021-05-24T23:04:50Z,UCgMJGv4cQl8-q71AyFeFmtg,ScreenCrush,1,2021-06-01T00:00:00Z,marvel announcement|eaternals breakdown|eatern...,631156,13310,399,1164,https://i.ytimg.com/vi/t-Y-1Ki6LEk/default.jpg,False,False,Marvel just unleashed their first trailer for ...,Reino Unido


In [24]:
# Verifica o tamanho do dataframe concatenado e a tipagem das colunas
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234723 entries, 0 to 234722
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           234723 non-null  object
 1   title              234723 non-null  object
 2   publishedAt        234723 non-null  object
 3   channelId          234723 non-null  object
 4   channelTitle       234723 non-null  object
 5   categoryId         234723 non-null  int64 
 6   trending_date      234723 non-null  object
 7   tags               234723 non-null  object
 8   view_count         234723 non-null  int64 
 9   likes              234723 non-null  int64 
 10  dislikes           234723 non-null  int64 
 11  comment_count      234723 non-null  int64 
 12  thumbnail_link     234723 non-null  object
 13  comments_disabled  234723 non-null  bool  
 14  ratings_disabled   234723 non-null  bool  
 15  description        231678 non-null  object
 16  country            2

In [25]:
# Recordando a tipagem do dataframe de categorias
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   kind                32 non-null     object
 1   etag                32 non-null     object
 2   id                  32 non-null     object
 3   snippet.title       32 non-null     object
 4   snippet.assignable  32 non-null     bool  
 5   snippet.channelId   32 non-null     object
dtypes: bool(1), object(5)
memory usage: 1.4+ KB


## Pré Processamento dos Dados

In [26]:
# Altera a tipagem do id do dataframe dos vídeos para possibilitar o merge com a tabela de categorias
df_csv['categoryId'] = df_csv['categoryId'].astype('str')

In [27]:
# Compara se alguma categoria do dataframes de video não está no dataframe de categorias
set(df_csv.categoryId.unique()).difference(set(df_json.id.unique()))

set()

In [28]:
# Compara se alguma categoria dataframe de categorias não está no dataframes de video, ou seja, não será utilizada no BI 
categorias_nao_aparecem = set(set(df_json.id.unique())).difference(df_csv.categoryId.unique())
print(f"{len(categorias_nao_aparecem)} categorias não estão presentes no dataframe de vídeos\
 e por isso serão dropadas. \nAlém de diminuir a dimensão da tabela, irá melhorar a experiência com o BI.")
# Filtragem apenas das categorias existentes no dataframe de videos
df_json = df_json[~df_json.id.isin(categorias_nao_aparecem)].copy()

17 categorias não estão presentes no dataframe de vídeos e por isso serão dropadas. 
Além de diminuir a dimensão da tabela, irá melhorar a experiência com o BI.


 - Todas as categorias do dataframe de vídeos foram encontradas no dataframe de categorias. Assim sendo, não haverá nenhum prejuízo na análise em decorrência dos arquivos JSON serem iguais.

In [29]:
# Transformando as colunas de data em datetime e removendo o horário
df_csv['publishedAt'] = pd.to_datetime(df_csv['publishedAt']).dt.date # Remove Hora, minuto, segundo
df_csv['publishedAt'] = pd.to_datetime(df_csv['publishedAt'], format='%y-%m-%d') # Transforma para data
df_csv['trending_date'] = pd.to_datetime(df_csv['trending_date']).dt.date # Remove Hora, minuto, segundo
df_csv['trending_date'] = pd.to_datetime(df_csv['trending_date'], format='%y-%m-%d') # Transforma para data

In [30]:
# Transformação das colunas booleanas e a coluna país em category para reduzir o uso de memoria do dataframe
df_csv['comments_disabled'] = df_csv['comments_disabled'].astype('category')
df_csv['ratings_disabled'] = df_csv['ratings_disabled'].astype('category')
df_csv['country'] = df_csv['country'].astype('category')

In [31]:
# Criação de coluna que booleana que mapeia vídeos comuns aos 4 países

# Localiza os vídeos que estão presentes na tabela de todos os países
videos_comuns_a_todos = df_csv.groupby('title').agg({'country':'nunique'}).query('country>3')
lista_videos_comuns_a_todos = list(videos_comuns_a_todos.index)
df_csv['todos'] = 'N'
df_csv.loc[df_csv.title.isin(lista_videos_comuns_a_todos), 'todos'] = 'S'
df_csv['todos'] = df_csv['todos'].astype('category')

In [32]:
# Avaliação do dataframe resultante após os pré-processamentos
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234723 entries, 0 to 234722
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   video_id           234723 non-null  object        
 1   title              234723 non-null  object        
 2   publishedAt        234723 non-null  datetime64[ns]
 3   channelId          234723 non-null  object        
 4   channelTitle       234723 non-null  object        
 5   categoryId         234723 non-null  object        
 6   trending_date      234723 non-null  datetime64[ns]
 7   tags               234723 non-null  object        
 8   view_count         234723 non-null  int64         
 9   likes              234723 non-null  int64         
 10  dislikes           234723 non-null  int64         
 11  comment_count      234723 non-null  int64         
 12  thumbnail_link     234723 non-null  object        
 13  comments_disabled  234723 non-null  category

### Análise de colunas para detecção de problemas que podem ocorrer no Power BI

In [33]:
# Analisando registros por colunas
for col in df_csv:
    print(df_csv[col].value_counts(), '\n')
    print(f'{"-" * 100}', '\n')

video_id
wY6UyatwVTA    48
pvPsJFRGleA    45
IKKbboquS9s    41
tQ0yjYUFKAE    40
QcS9ZndErHc    39
               ..
t7TRtf-6hIo     1
koqnU50NMxk     1
Pyypif-xtK8     1
Kn5ntXJJleU     1
ayE5rF-BzeE     1
Name: count, Length: 27061, dtype: int64 

---------------------------------------------------------------------------------------------------- 

title
Starlink Mission                                                                                   216
We Broke Up                                                                                         58
India claim stunning series win, end Australia's Gabba streak | Vodafone Test Series 2020-21        48
we broke up                                                                                         48
Justin Bieber - Holy ft. Chance The Rapper                                                          45
                                                                                                  ... 
what if to do this... #

In [34]:
# Avaliando Ids com mais de um título
df_csv.groupby('channelId').agg({'channelTitle':'nunique'}).query('channelTitle>1').sort_values('channelTitle', ascending=False)

,channelTitle
channelId,
UCQG-g3mLDliBH180J6nmqKg,5
UC-2XiVy5AbEn2TXf0f2APeQ,3
UC9APf0VQtg1VriRFjk9RfFQ,3
UCj34AOIMl_k1fF7hcBkD_dw,3
UCVepIJedg6BMQeSgjPY4e6A,3
...,...
UCz7sPJrTGM3x41_RnptajyQ,2
UCzCutgYFZpBdElMw3krjG0A,2
UCzwFZOhCfCZTgLruMz_RWCw,2


In [35]:
df_csv.groupby('channelTitle').agg({'channelId':'nunique'}).query('channelId>1')

,channelId
channelTitle,
CONMEBOL Libertadores,2
ESPN Brasil,2
GOT7,2
HanRidge,2
MTV,2
SML,2
THE BOYZ,2
TODAY,2
Various Artists - Topic,8


In [36]:
# Ids de canais com mais de um título
ids_mais_titulo = df_csv.groupby('channelId').agg({'channelTitle':'nunique'}).query('channelTitle>1').index

n_titulos = df_csv[df_csv['channelId'].isin(ids_mais_titulo)].channelTitle.nunique()

print(f"{len(ids_mais_titulo)} Ids de canais, possuem mais de um título associado,\
 isso significa que 'channelId' não é uma chave primária.\nEsses {len(ids_mais_titulo)} Ids possuem\
 {n_titulos} títulos associados.")

106 Ids de canais, possuem mais de um título associado, isso significa que 'channelId' não é uma chave primária.
Esses 106 Ids possuem 220 títulos associados.


In [37]:
# Títulos de canais com mais de um Id
titulos_mais_ids = df_csv.groupby('channelTitle').agg({'channelId':'nunique'}).query('channelId>1').index

n_ids = df_csv[df_csv['channelTitle'].isin(titulos_mais_ids)].channelId.nunique()

print(f"{len(titulos_mais_ids)} títulos de canais, possuem mais de um Id,\
 isso significa que 'channelTitle' também não é uma chave primária.\nEsses {len(titulos_mais_ids)} títulos possuem\
 {n_ids} Ids associados.")

11 títulos de canais, possuem mais de um Id, isso significa que 'channelTitle' também não é uma chave primária.
Esses 11 títulos possuem 28 Ids associados.


In [38]:
# Ids de videos com mais de um título
ids_mais_titulo = df_csv.groupby('video_id').agg({'title':'nunique'}).query('title>1').index

n_titulos = df_csv[df_csv['video_id'].isin(ids_mais_titulo)].title.nunique()

print(f"{len(ids_mais_titulo)} Ids de vídeos, possuem mais de um título associado,\
 isso significa que 'video_id' não é uma chave primária.\nEsses {len(ids_mais_titulo)} Ids possuem\
 {n_titulos} títulos associados.")

546 Ids de vídeos, possuem mais de um título associado, isso significa que 'video_id' não é uma chave primária.
Esses 546 Ids possuem 1146 títulos associados.


In [39]:
# Títulos de vídeos com mais de um Id
titulos_mais_ids = df_csv.groupby('title').agg({'video_id':'nunique'}).query('video_id>1').index

n_ids = df_csv[df_csv['title'].isin(titulos_mais_ids)].video_id.nunique()

print(f"{len(titulos_mais_ids)} títulos de vídeos, possuem mais de um Id,\
 isso significa que 'title' não é uma chave primária.\nEsses {len(titulos_mais_ids)} títulos possuem\
 {n_ids} Ids associados.")

59 títulos de vídeos, possuem mais de um Id, isso significa que 'title' não é uma chave primária.
Esses 59 títulos possuem 151 Ids associados.


In [40]:
# Verificação de duplicações
df_csv[df_csv.duplicated()]

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,country,todos
8299,OUZgxTQYST0,Live - Chiclete com Banana - Vintage,2020-09-20,UCKY0ExOhG86FyDmtr_aM6dg,Chiclete com Banana,10,2020-09-22,[None],183939,3439,113,69,https://i.ytimg.com/vi/OUZgxTQYST0/default.jpg,False,False,Live - Chiclete com Banana - Vintage#ChicleteC...,Brasil,N
38398,92MOgwbgG7U,500 SEGREDOS DO MINECRAFT QUE VOCÊ NÃO SABIA (...,2021-02-22,UCEfGV5hx2VrXl4jOUnG0MRQ,Geleia,20,2021-02-24,minecraft|curiosidades minecraft|geleia minecr...,823585,48015,1300,1272,https://i.ytimg.com/vi/92MOgwbgG7U/default.jpg,False,False,📷 MEU INSTAGRAM: https://www.instagram.com/olu...,Brasil,N
38424,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-02-24,roda viva|tv cultura|debate|jornalismo|entrevi...,35417,3349,439,366,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
38426,wmZEliQOsDc,NOOB NO CÉU | PARAFERNALHA,2021-02-22,UCddYq41_tZ1FnLlguLT6-Ow,Parafernalha,24,2021-02-24,parafernalha|parafernalia|humor|comedia|esquet...,140859,19409,355,348,https://i.ytimg.com/vi/wmZEliQOsDc/default.jpg,False,False,Samuca é um noob que bateu as botas e foi para...,Brasil,N
38427,X5SxRpbhMxM,Live de atualizações coronavirus (22/02),2021-02-22,UCHVFMFyiU1DuJ13ybBonW9g,Governo do Rio Grande do Sul,24,2021-02-24,[None],60712,529,151,123,https://i.ytimg.com/vi/X5SxRpbhMxM/default.jpg,False,False,Inscreva-se no canal!,Brasil,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216117,qwcDqoTT6Xo,Handforth Parish Council Planning Meeting foll...,2021-02-17,UCvOOP3STGZlLUSuTHdRtF-A,Handforth Parish Council,24,2021-02-24,[None],105301,911,77,677,https://i.ytimg.com/vi/qwcDqoTT6Xo/default.jpg,False,False,NaN,Reino Unido,N
216118,QBFiiEVBWvE,NASA Mars 2020 Perseverance Rover Landing! 🔴 Live,2021-02-18,UCdxb3hRFmbaMCo8OZc6OCuQ,Overlook Horizon,28,2021-02-24,nasa|mars rover|mars perseverance|nasa rover|m...,96555,1819,37,49,https://i.ytimg.com/vi/QBFiiEVBWvE/default.jpg,False,False,Mars 2020 is a Mars rover mission by NASA's Ma...,Reino Unido,N
216120,br_cKwfR8gI,Pyra / Mythra in Smash!!! Nice!!!,2021-02-17,UC2gB7YCgfHIskI3HsY1teDw,Alpharad Deluxe,20,2021-02-24,[None],853461,41811,1278,5409,https://i.ytimg.com/vi/br_cKwfR8gI/default.jpg,False,False,neat#SmashBros #Reaction #Episode1,Reino Unido,N
216121,u613yk4RGGw,Driving My Wrecked Mclaren 675LT For The First...,2021-02-17,UCvAhDxNNUDhi78tMXVGBUaQ,Tavarish,2,2021-02-24,aston martin|mercedes|amg|hyundai|bmw|lamborgh...,1061571,41331,817,2337,https://i.ytimg.com/vi/u613yk4RGGw/default.jpg,False,False,Find the world's BEST DEALS on cars with AutoT...,Reino Unido,N


In [41]:
# Escolha aleatória de um registro para análise
df_csv[df_csv.title == 'Roda Viva | Teresa Cristina | 22/02/2021'].sort_values(['likes','dislikes'])

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,country,todos
38224,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-02-24,roda viva|tv cultura|debate|jornalismo|entrevi...,35417,3349,439,366,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
38424,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-02-24,roda viva|tv cultura|debate|jornalismo|entrevi...,35417,3349,439,366,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
38658,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-02-25,roda viva|tv cultura|debate|jornalismo|entrevi...,46778,3997,617,624,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
38868,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-02-26,roda viva|tv cultura|debate|jornalismo|entrevi...,50216,4197,653,688,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
39074,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-02-27,roda viva|tv cultura|debate|jornalismo|entrevi...,52485,4324,666,719,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
39337,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-03-01,roda viva|tv cultura|debate|jornalismo|entrevi...,57008,4589,699,760,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
39563,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-03-02,roda viva|tv cultura|debate|jornalismo|entrevi...,58083,4636,708,774,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N
39770,YBemdKjn59E,Roda Viva | Teresa Cristina | 22/02/2021,2021-02-23,UCNVsZnDXOM4PodYIEgM2e4w,Roda Viva,25,2021-03-02,roda viva|tv cultura|debate|jornalismo|entrevi...,58175,4646,709,778,https://i.ytimg.com/vi/YBemdKjn59E/default.jpg,False,False,"No Roda Viva, a jornalista Vera Magalhães rece...",Brasil,N


In [42]:
# Escolha aleatória de outro registro para análise
df_csv[df_csv.title == 'Live - Chiclete com Banana - Vintage'].sort_values(['likes','dislikes'])

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,country,todos
7806,OUZgxTQYST0,Live - Chiclete com Banana - Vintage,2020-09-20,UCKY0ExOhG86FyDmtr_aM6dg,Chiclete com Banana,10,2020-09-20,[None],164559,3182,73,36,https://i.ytimg.com/vi/OUZgxTQYST0/default.jpg,False,False,Live - Chiclete com Banana - Vintage#ChicleteC...,Brasil,N
8534,OUZgxTQYST0,Live - Chiclete com Banana - Vintage,2020-09-20,UCKY0ExOhG86FyDmtr_aM6dg,Chiclete com Banana,10,2020-09-23,[None],185172,3439,111,70,https://i.ytimg.com/vi/OUZgxTQYST0/default.jpg,False,False,Live - Chiclete com Banana - Vintage#ChicleteC...,Brasil,N
8298,OUZgxTQYST0,Live - Chiclete com Banana - Vintage,2020-09-20,UCKY0ExOhG86FyDmtr_aM6dg,Chiclete com Banana,10,2020-09-22,[None],183939,3439,113,69,https://i.ytimg.com/vi/OUZgxTQYST0/default.jpg,False,False,Live - Chiclete com Banana - Vintage#ChicleteC...,Brasil,N
8299,OUZgxTQYST0,Live - Chiclete com Banana - Vintage,2020-09-20,UCKY0ExOhG86FyDmtr_aM6dg,Chiclete com Banana,10,2020-09-22,[None],183939,3439,113,69,https://i.ytimg.com/vi/OUZgxTQYST0/default.jpg,False,False,Live - Chiclete com Banana - Vintage#ChicleteC...,Brasil,N
8775,OUZgxTQYST0,Live - Chiclete com Banana - Vintage,2020-09-20,UCKY0ExOhG86FyDmtr_aM6dg,Chiclete com Banana,10,2020-09-24,[None],186115,3475,111,75,https://i.ytimg.com/vi/OUZgxTQYST0/default.jpg,False,False,Live - Chiclete com Banana - Vintage#ChicleteC...,Brasil,N


 - _Nas amostras as duplicações ficaram evidentes, será feito o drop_duplicates para remover futuras distorções das análises._

In [43]:
df_csv.drop_duplicates(inplace=True)

### Análise Descritiva do DataFrame de Vídeos

In [44]:
# Configurar as opções de exibição para notação numérica normal
pd.set_option('display.float_format', lambda x: '%.0f' % x)

df_csv.describe(percentiles=[0.25,0.50,0.75,0.90,0.95,0.99])

,publishedAt,trending_date,view_count,likes,dislikes,comment_count
count,234318,234318,234318,234318,234318,234318
mean,2021-01-04 00:59:30.409443328,2021-01-08 01:06:50.672675328,2274622,136694,2853,13538
min,2020-07-27 00:00:00,2020-08-12 00:00:00,0,0,0,0
25%,2020-10-20 00:00:00,2020-10-24 00:00:00,342668,15502,260,1059
50%,2021-01-04 00:00:00,2021-01-09 00:00:00,812326,43137,678,2854
75%,2021-03-19 00:00:00,2021-03-24 00:00:00,1958411,116349,1928,7854
90%,2021-05-03 00:00:00,2021-05-07 00:00:00,4694064,284208,5171,20513
95%,2021-05-17 00:00:00,2021-05-21 00:00:00,8256343,493615,10372,38902
99%,2021-06-05 00:00:00,2021-06-11 00:00:00,25712309,1600780,35983,146787
max,2021-06-15 00:00:00,2021-06-16 00:00:00,278080610,16213756,879357,6817450


### Geração dos arquivos pré-processados no python
 - _Os arquivos serão salvos no formato parquet para otimizar a utilização de espaço de armazenamento._

In [42]:
df_csv.to_parquet('youtube_videos.parquet', index=False)
df_json.to_parquet('categorias.parquet', index=False)

**As próximas etapas serão executadas no ``Power BI``**

[![Acesse o Dashboard](https://img.shields.io/badge/PowerBI-F2C811?style=for-the-badge&logo=Power%20BI&logoColor=white)](https://app.powerbi.com/view?r=eyJrIjoiZmFiZTNmNmYtOWQwMy00Mzc0LThhMzgtYmEzNjI0MjMwNDM3IiwidCI6ImRlYTcxOWY4LWZmODEtNDRlZi1hMjljLTIyMmMxMGYyZmVhYSJ9)